# Yearly Ridership Data Visualization And Analysis For The Taipei Metro System In Taiwan.

## Section 1. Aggregate Data
##### From Different Data Sources

### Steps
1. Retrieve coordinates from shapefiles
2. Check the data (look for missing data)
3. Add new station names and coordinates when needed (Enter missing data)
4. Retrieve ridership data from excel tables or CSV files
5. Aggregate data (i.e. daily to monthly, yearly...)
6. Put data together
7. Check the result
8. Export data to desired data format (such as JSON, GeoJSON...)


In [66]:
# import module
import geopandas as gpd
import pandas as pd


In [67]:
# read shapefile to extract coordinates
df_metro_shap = gpd.read_file('D:\上課資料\Geog778\Project\Data\Processed\MRT_TPE_Station_Modified\TpeMrtStations_TWD97_Unicode.shp')
# TWD97 => epsg=3824 (degree)
df_metro_shap_geo = df_metro_shap.to_crs(epsg=3824)

# create a coordinate dictionary
dic_coord= {}
stat_num = len(df_metro_shap_geo)
for i in range(stat_num):
    dic_coord[df_metro_shap_geo['EngName'][i]] = [df_metro_shap_geo['geometry'][i].x, df_metro_shap_geo['geometry'][i].y]

# add missing coordinates (Dingpu)
dic_coord['Dingpu'] = [121.41927, 24.95975]
# fix inaccurate coordinates
dic_coord['BL-Banqiao'] = [121.46242, 25.01409]
dic_coord['Y-Banqiao'] = [121.46469, 25.01554]
dic_coord['Xinpu Minsheng'] = [121.46697, 25.02630]
# 119 stations in total

In [68]:
# create year list
year_lst= list(range(2015,2023))
dic_sum = {}
year_num = len(year_lst)
dic_chg_p = {}

# create station name list
station_df = pd.read_excel("D:/上課資料/Geog778/Project/Data/Processed/Ridership_{year}.xlsx".format(year= str(max(year_lst))), usecols="B:DP")
station_name = []
for col in station_df:
    station_name.append(col)

name_num = len(station_name)

In [69]:
csv_lines = []
csvHeader = ['year', 'stationName', 'ridership', 'longitude', 'latitude', 'changeP']

# read all yearly ridership sum
for i in range(year_num):
    df_sum = pd.read_excel("D:/上課資料/Geog778/Project/Data/Processed/Ridership_{year}.xlsx".format(year= str(year_lst[i])), usecols="B:DP").sum()
    sum_lst = []
    for num in df_sum:
        sum_lst.append(round(num))
    dic_sum[str(year_lst[i])]= sum_lst
    
    if i == year_num-1:
        while i >= 1:
            chg_sum=[]
            for j in range(name_num):
                if dic_sum[str(year_lst[i-1])][j] == 0:
                    chg_p = 'No Data For previous year'
                    chg_sum.append(chg_p)
                else:
                    chg_p = (dic_sum[str(year_lst[i])][j]-dic_sum[str(year_lst[i-1])][j])/dic_sum[str(year_lst[i-1])][j]*100
                    chg_p = round(chg_p, 2)
                    chg_p = str(chg_p) + "%"
                    chg_sum.append(chg_p)
            dic_chg_p[str(year_lst[i])] = chg_sum
            i -= 1
        
        if i == 0:
            chg_sum=[]
            for j in range(name_num):
                chg_p = 'No Data For previous year'
                chg_sum.append(chg_p)
            dic_chg_p[str(year_lst[i])] = chg_sum
        break

    for (num, st_name) in zip(sum_lst, station_name):
        # row: year, stationName, ridership, longitude, latitude
        row = [year_lst[i], st_name, num, round(dic_coord[st_name][0],5), round(dic_coord[st_name][1],5)]
        csv_lines.append(row)

for line in csv_lines:
    chg_p = dic_chg_p[str(line[0])][station_name.index(line[1])]
    line.append(chg_p)

In [70]:
# If want to export the data to CSV format, activate this cell
import csv 
# create a empty text file to receive data
with open("ridership.csv", 'w+', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(csv_lines)

In [71]:
# create a new dataframe to contain data
df_yearlyRid = pd.DataFrame()

# add station names
df_yearlyRid['stationName'] = station_name

# add ridership data
for year in year_lst:
    df_yearlyRid['yr'+str(year)] = dic_sum[str(year)] 
    df_yearlyRid['chgP'+str(year)] = dic_chg_p[str(year)]

# add coordinates and change percentage
lng = []
lat = []
for i in range(name_num):
    lng.append(round(dic_coord[df_yearlyRid['stationName'][i]][0], 5))
    lat.append(round(dic_coord[df_yearlyRid['stationName'][i]][1], 5))
df_yearlyRid['longitude']=lng
df_yearlyRid['latitude']=lat
df_yearlyRid


,stationName,yr2015,chgP2015,yr2016,chgP2016,yr2017,chgP2017,yr2018,chgP2018,yr2019,chgP2019,yr2020,chgP2020,yr2021,chgP2021,yr2022,chgP2022,longitude,latitude
0,Songshan Airport,1996920,No Data For previous year,2092136,4.77%,2019020,-3.49%,2033526,0.72%,2088370,2.7%,1445689,-30.77%,969124,-32.96%,1125624,16.15%,121.55164,25.06310
1,Zhongshan Junior High School,5264138,No Data For previous year,5256347,-0.15%,5160555,-1.82%,5134302,-0.51%,5121762,-0.24%,4530029,-11.55%,3428641,-24.31%,3701648,7.96%,121.54422,25.06081
2,Nanjing Fuxing,11111846,No Data For previous year,12013775,8.12%,12601508,4.89%,13322009,5.72%,13782338,3.46%,12528536,-9.1%,9553176,-23.75%,10749968,12.53%,121.54404,25.05229
3,Zhongxiao Fuxing,18324235,No Data For previous year,18469890,0.79%,17883808,-3.17%,17795928,-0.49%,17588313,-1.17%,14802750,-15.84%,11101659,-25.0%,12028310,8.35%,121.54379,25.04160
4,Daan,7454570,No Data For previous year,7923790,6.29%,8232428,3.9%,8631916,4.85%,8877145,2.84%,7901826,-10.99%,5961757,-24.55%,6480952,8.71%,121.54363,25.03282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Banxin,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,768557,No Data For previous year,718162,-6.56%,852545,18.71%,121.47293,25.01400
115,Y-Banqiao,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,2599601,No Data For previous year,2197512,-15.47%,2616227,19.05%,121.46469,25.01554
116,Xinpu Minsheng,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,709877,No Data For previous year,598946,-15.63%,675441,12.77%,121.46697,25.02630
117,Xingfu,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,0,No Data For previous year,1066536,No Data For previous year,983128,-7.82%,1143045,16.27%,121.45990,25.04978


In [72]:
# convert dataframe to GeoJSON
gdf_yearlyRid = gpd.GeoDataFrame(df_yearlyRid, geometry=gpd.points_from_xy(df_yearlyRid.longitude, df_yearlyRid.latitude))

# save as GeoJSON and export the data
yearlyRid_GeoJ= gdf_yearlyRid.to_file('D:/上課資料/Geog778/Project/Data/Processed/ridership.geojson', driver='GeoJSON')
yearlyRid_GeoJ

c:\KC\anaconda3\envs\KC_Geog573\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Section 2. Data Visualization with Pydeck
### Steps
1. Read the data
2. Calculate ridership for color ranking (High, Medium, Low)
3. Scaled down number range for visualization
4. Set initial view for the map
5. Create map layer
6. Customize the texts displayed (Clean up unnecessary content)
7. Create widget for controlling the map
8. Connect widget and the map
9. Display the map


In [73]:
import pydeck
import ipywidgets

In [74]:
# data
tpeMRTridership = pd.DataFrame(csv_lines, columns=csvHeader)

# using the first and the third quantiles for ranking
quantiles = tpeMRTridership.ridership.quantile([0.25, 0.5, 0.75])
q_1 = quantiles[0.25]
q_3 = quantiles[0.75]

# scaled down the ridership for visualization
tpeMRTridership['scaled_ridership'] = tpeMRTridership['ridership']/100_000
df_leng = len(tpeMRTridership)
color_set = []
for i in range(df_leng):
    if(tpeMRTridership['ridership'][i] >= q_3):
        color_set.append(0.9)
    elif(tpeMRTridership['ridership'][i] >= q_1):
        color_set.append(0.5)
    elif(tpeMRTridership['ridership'][i] ==0 ):
        color_set.append(0)
    else:
        color_set.append(0.3)
tpeMRTridership['colorRank'] =color_set

# set initial view
view = pydeck.ViewState(latitude=25.05, longitude=121.50, pitch=30, zoom=9)

# layer
tpeMRTridership_layer = pydeck.Layer('ColumnLayer',
                             data=tpeMRTridership,
                             get_position=['longitude', 'latitude'],
                             get_elevation='scaled_ridership',
                             elevation_scale=100,
                             radius=300,
                             get_fill_color=['255', '255*colorRank', '200*colorRank', '150'],
                             pickable=True,
                             auto_highlight=True)

# Customize the texts displayed
tooltip = {
    'html': """
    <p><b>Name:</b> {stationName}</p>
    <p><b>Coordinate:</b> {longitude}, {latitude}</p>
    <p><b>Year:</b> {year}</p>
    <p><b>Ridership:</b> {ridership}</p>
    <p><b>Ridership Change Percentage:</b> {changeP}</p>
    """
}


# render map
# with no map_style, map goes to default
tpeMRTridership_layer_map = pydeck.Deck(layers=tpeMRTridership_layer, 
                               initial_view_state=view,
                               tooltip=tooltip
                               )

# widgets
slider = ipywidgets.IntSlider(value=2015, min=2015, max=2022, step=1)
play = ipywidgets.Play(value=2015, min=2015, max=2022, step=1, description='Press play', interval=1_000)
ipywidgets.jslink((play, 'value'), (slider, 'value'))
layout = ipywidgets.HBox([slider, play])

# widgets function
def update_plot(year):
    tpeMRTridership_layer.data = tpeMRTridership[tpeMRTridership['year'] == year]
    return tpeMRTridership_layer_map.update()

# interaction between widget and function
interact = ipywidgets.interactive_output(update_plot, {'year': slider})


In [75]:
# display the map and the widget
print("Legend")
print("Light Yellow: High")
print("Orange: Medium")
print("Red: Low")
display(layout, interact)
tpeMRTridership_layer_map.show()


Legend
Light Yellow: High
Orange: Medium
Red: Low


Output()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…